## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

fileName = 'jet.bmp'
url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
jet_img_gs = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
jet_hist_gs = cv2.calcHist([jet_img_gs], [0], None, [256], [0,256])

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].imshow(jet_img_gs, 'gray')
ax[1].plot(jet_hist_gs)
ax[1].set_xlim([0, 256])
plt.show()

In [ ]:
#BBHE
brightness = np.mean(jet_img_gs)
brightness = int(np.floor(brightness))
print("Average brightness: ", brightness)
# 4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
jet_hist_left, jet_hist_right= jet_hist_gs[:brightness], jet_hist_gs[brightness:]
# Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
# Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
jet_hist_left_cumsum, jet_hist_right_cumsum = np.cumsum(jet_hist_left), np.cumsum(jet_hist_right)
jet_hist_left_cumsum, jet_hist_right_cumsum = jet_hist_left_cumsum / np.max(jet_hist_left_cumsum), jet_hist_right_cumsum / np.max(jet_hist_right_cumsum)
# 5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
# Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
# Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
jet_hist_left_cumsum = jet_hist_left_cumsum * brightness
# Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
jet_hist_right_cumsum = brightness + 1 + (jet_hist_right_cumsum * (255 - brightness - 1))
# $C_{1n} = (lm)*C1;$<br>
# $C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
# Następnie dwie części tablicy przekodowań należy połączyć.
lut = np.concatenate((jet_hist_left_cumsum, jet_hist_right_cumsum))
# 6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
jet_img_gs_bbhe = cv2.LUT(jet_img_gs, lut)
# Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:

_, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].imshow(jet_img_gs_bbhe, 'gray')
ax[0].set_title('BBHE image')
ax[1].plot(cv2.calcHist([jet_img_gs_bbhe], [0], None, [256], [0,256]))
ax[1].set_title('BBHE histogram')

plt.show()

In [ ]:
img_he_jet = cv2.equalizeHist(jet_img_gs)

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 5))
ax[0].imshow(img_he_jet, 'gray')
ax[0].set_title("HE image")
ax[1].plot(cv2.calcHist([img_he_jet], [0], None, [256], [0,256]))
ax[1].set_title("HE histogram")
plt.show()